In [120]:
import os
import pickle
import pandas as pd

from bert_score import score as bert_score_fn
import pickle
import evaluate
import string

In [121]:
not_present_dic={"age": "no.","history":"no history was found.","parenchymal_distortion":"no.","nodules_echo_size":"unknown."}

In [122]:
model="biogptQA2e55e66no_copydescription8batchfinal"
questions=["age","history","parenchymal_distortion","nodules_echo_size"]
# questions=["age","tipo","tecnica","family","history","symtomatic",
#            "prosthesis","birads","density_mammo","calcifications_benign","density_echo","ganglio_mamo","lymph_benign","lymph_suspicious","parenchymal_distortion","simple_cyst","ductal_ectasia",
#           "nodules_echo_num","nodules_echo_description","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_location","nodules_echo_size","nodules_echo_known","nodules_echo_stable"]

In [123]:
def truncate_before_keyword(text, keyword):
    """Truncate text before keyword (e.g., 'comparative study') and end with a period."""
    keyword_pos = text.lower().find(keyword.lower())
    if keyword_pos == -1:
        cleaned = text.strip()
    else:
        # Text before keyword
        partial = text[:keyword_pos]
        # Find last punctuation before keyword
        match = re.search(r'(.+?)[\.\,\;\n][^.,;\n]*$', partial)
        cleaned = match.group(1).strip() if match else partial.strip()
    
    # Ensure the result ends with a single period
    if not cleaned.endswith('.'):
        cleaned += '.'
    
    return cleaned

In [124]:
def calculate_acc(pred,truth,tipo,point=False):
    acc=0
    total=0
    for key,real in truth.items():
        if key in pred:
            total+=1
            predicted=pred[key]   
            
            if predicted.startswith(not_present_dic[tipo]):
                predicted="no response."
            elif tipo=="nodules_echo_size":
                predicted = predicted.split("mm")[0] + "mm."
            else:
                predicted = predicted.split(".")[0] + "."
            predicted=truncate_before_keyword(predicted, "comparative study")
            if predicted.lower()==str(real).lower():
                acc+=1
            else:
                print(key,"REAL: ", real,"PREDICTED: ",predicted)
        else:
            print(key)
    print(acc)
    print(total)

    return acc/total

In [128]:
import re
import string
import evaluate

rouge = evaluate.load("rouge")
exact_match = evaluate.load("exact_match")

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = set(prediction_tokens) & set(ground_truth_tokens)
    num_same = len(common)
    if len(prediction_tokens) == 0 or len(ground_truth_tokens) == 0:
        return int(prediction_tokens == ground_truth_tokens)
    if num_same == 0:
        return 0
    precision = num_same / len(prediction_tokens)
    recall = num_same / len(ground_truth_tokens)
    return 2 * precision * recall / (precision + recall)

def calculate_metrics(pred, truth, tipo):
    preds, refs = [], []
    for key, real in truth.items():
        if key not in pred:
            continue
        predicted = pred[key]
        if tipo in not_present_dic:
            if predicted.startswith(not_present_dic[tipo]):
                predicted="no response."
            elif tipo=="nodules_echo_size":
                predicted = predicted.split("mm")[0] + "mm."
            else:
                predicted = predicted.split(".")[0] + "."
        
        predicted=truncate_before_keyword(predicted, "comparative study")
        preds.append(predicted)
        refs.append(real)
    print(preds,refs)
    # Compute metrics
    f1s, ems = [], []
    for p, r in zip(preds, refs):
        f1s.append(f1_score(p, r))
        ems.append(int(normalize_answer(p) == normalize_answer(r)))


    rouge_scores = rouge.compute(predictions=preds, references=refs)
    rouge_l = rouge_scores["rougeL"]

    P, R, F1 = bert_score_fn(preds, refs, lang="en", verbose=False, rescale_with_baseline=True)
    print(F1)
    bert_f1 = F1.mean().item()
    bert_R = R.mean().item()

    print(f"Examples: {len(preds)}")
    print(f"Token F1: {sum(f1s)/len(f1s):.4f}")
    print(f"Exact Match: {sum(ems)/len(ems):.4f}")
    print(f"ROUGE-L: {rouge_l:.4f}")
    print(f"BERTScore F1: {bert_f1:.4f}")
    print(f"BERTScore R: {bert_R:.4f}")

    return {
        "token_f1": sum(f1s)/len(f1s),
        "exact_match": sum(ems)/len(ems),
        "rouge_l": rouge_l,
        "bert_score_f1": bert_f1,
        "bert_score_R": bert_R,
        "list_bert": F1,
    }

In [126]:
not_present_dic.values()

dict_values(['no.', 'no history was found.', 'no.', 'unknown.'])

In [137]:
model="biogpt"
f1_total=[]
for tipo in questions:
    print(tipo.upper())
    with open(f"test_results/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    if tipo=="parenchymal_distortion":
        truth["547-205-006-20220425-132446_parenchymal_distortion"]="post-surgical changes in right breast."
    truth = {
    k: ("no response." if v in list(not_present_dic.values()) else v)
    for k, v in truth.items()
    }
        
    with open(f"test_results/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point="mm"))
    else:
        print(calculate_acc(output,truth,tipo))

    result=calculate_metrics(output, truth, tipo)
    f1_bio=result["list_bert"]
    if tipo!="age":
        f1_total+=f1_bio

AGE
70
70
1.0
['no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', '50.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', '77.', 'no response.', '28.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', '51.', 'no response.', '50.', 'no response.', '59.', 'no response.', '53.', 'no response.', '50.', 'no response.', 'no response.', '43.', 'no response.', 'no response.', 'no response.', 'no response.', '44.', '44.', 'no response.', 'no response.', 'no response

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
Examples: 70
Token F1: 1.0000
Exact Match: 1.0000
ROUGE-L: 1.0000
BERTScore F1: 1.0000
BERTScore R: 1.0000
HISTORY
424-677-160-20220620-121757_history REAL:  no response. PREDICTED:  History of mother with breast cancer at 63 years of age.
466-779-284-20220314-135324_history REAL:  no response. PREDICTED:  second-degree history of breast cancer (paternal grandmother with m

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        0.1677, 1.0000, 1.0000, 0.0019, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.2638, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.8763, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        0.3209, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])
Examples: 64
Token F1: 0.9245
Exact Match: 0.9219
ROUGE-L: 0.9432
BERTScore F1: 0.9474
BERTScore R: 0.9493
PARENCHYMAL_DISTORTION
493-946-873-20240201-105209_parenchymal_distortion REAL:  the breast parenchyma shows areas of architectural distortion in the right upper outer quadrant breast adjacent to the coil. PREDICTED:  architectural distortion in the right upper outer quadrant breast adjacent to the coil that I cons

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.4092, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.8641, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.5016, 0.1649, 1.0000,
        1.0000, 1.0000, 0.2520, 1.0000, 1.0000, 1.0000, 1.0000, 0.0623, 1.0000,
        1.0000])
Examples: 64
Token F1: 0.9418
Exact Match: 0.9219
ROUGE-L: 0.9447
BERTScore F1: 0.9415
BERTScore R: 0.9441
NODULES_ECHO_SIZE
398-860-253-20231102-182443_nodules_echo_size_1 REAL:  11mm. PREDICTED:  12mm.
612-418-370-20231102-192916_nodules_echo_size_1 REAL:  1cm. PREDICTED:  8 mm.
674-466-656-20240110-111525_nodules_echo_size_1 REAL:  20x14 mm. PREDICTED:  9mm.
764-050-113-20231031-121931.719_nodules_echo_size_1 REAL:  

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9951, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.8368, 1.0000, 0.3967, 1.0000, 1.0000, 0.9867,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.4890])
Examples: 25
Token F1: 0.7833
Exact Match: 0.8000
ROUGE-L: 0.8000
BERTScore F1: 0.9482
BERTScore R: 0.9478


In [138]:
[round(v.item(),4) for v in f1_total]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1677,
 1.0,
 1.0,
 0.0019,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2638,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8763,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.3209,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4092,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8641,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5016,
 0.1649,
 1.0,
 1.0,
 1.0,
 0.252,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0623,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9951,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8368,
 1.0,
 0.3967,
 1.0,
 1.0,
 0.9867,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.489]

In [130]:
f1_total

[tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.),
 tensor(1.0000),
 tensor(1.),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.),
 tensor(1.0000),
 tensor(1.),
 tensor(1.0000),
 tensor(1.0000),
 tensor(1.),
 tensor(1.0000),
 tensor(

In [107]:
model="clinicalt5_tokenized"
for tipo in questions:
    print(tipo.upper())
    with open(f"test_results/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    if tipo=="parenchymal_distortion":
        truth["547-205-006-20220425-132446_parenchymal_distortion"]="post-surgical changes in right breast."
    truth = {
    k: ("no response." if v in list(not_present_dic.values()) else v)
    for k, v in truth.items()
    }
        
    with open(f"test_results/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,tipo,point="mm"))
    else:
        print(calculate_acc(output,truth,tipo))

    calculate_metrics(output, truth, tipo)

AGE
70
70
1.0
['no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', '50.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', '77.', 'no response.', '28.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', 'no response.', '51.', 'no response.', '50.', 'no response.', '59.', 'no response.', '53.', 'no response.', '50.', 'no response.', 'no response.', '43.', 'no response.', 'no response.', 'no response.', 'no response.', '44.', '44.', 'no response.', 'no response.', 'no response

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 70
Token F1: 1.0000
Exact Match: 1.0000
ROUGE-L: 1.0000
BERTScore F1: 1.0000
BERTScore R: 1.0000
HISTORY
212-130-033-20230313-153938_history REAL:  no response. PREDICTED:  yes was found.
259-466-829-20230717-131725.078000_history REAL:  no response. PREDICTED:  yes was found.
508-342-622-20221207-134240_history REAL:  no response. PREDICTED:  yes was found.
543-693-696-20220905-113459.250000_history REAL:  no response. PREDICTED:  yes was found.
547-205-006-20220425-132446_history REAL:  history of surgery in the right breast with histological result of multiple epithelial proliferative lesions, without signs of malignancy in 2020. PREDICTED:  history of surgery in the right breast with histological result of benignity in right breast.
569-988-603-20230227-132814.328000_history REAL:  no response. PREDICTED:  History of breast cancer: second degree, maternal line and postmenopausal aunt.
612-418-370-20231102-192916_history REAL:  no response. PREDICTED:  koik was found.
740-

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 64
Token F1: 0.8497
Exact Match: 0.8281
ROUGE-L: 0.8694
BERTScore F1: 0.8962
BERTScore R: 0.8993
PARENCHYMAL_DISTORTION
326-065-308-20220620-124632_parenchymal_distortion REAL:  asymmetry of the density in the lower inner quadrant of the right breast that with tomosynthesis is dissociated and appears to be related to an accumulation of fibroglandular tissue. PREDICTED:  asymmetry of the density in the lower inner quadrant of the right breast that with tomosynthesis is dissociated and appears related to an accumulation of fibroglandular tissue.
493-946-873-20240201-105209_parenchymal_distortion REAL:  the breast parenchyma shows areas of architectural distortion in the right upper outer quadrant breast adjacent to the coil. PREDICTED:  no response.
621-280-972-20240124-112541_parenchymal_distortion REAL:  post-treatment changes in the right breast . PREDICTED:  post-treatment changes in the right breast.
691-685-245-20230717-124802.687000_parenchymal_distortion REAL:  no respo

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 64
Token F1: 0.9374
Exact Match: 0.9062
ROUGE-L: 0.9425
BERTScore F1: 0.9424
BERTScore R: 0.9369
NODULES_ECHO_SIZE
612-418-370-20231102-192916_nodules_echo_size_1 REAL:  1cm. PREDICTED:  1cm.mm.
674-466-656-20240110-111525_nodules_echo_size_1 REAL:  20x14 mm. PREDICTED:  9mm.
764-050-113-20231031-121931.719_nodules_echo_size_1 REAL:  5mm. PREDICTED:  12mm.
978-326-059-20231115-183229.962_nodules_echo_size_1 REAL:  3 cm. PREDICTED:  3 cm.mm.
21
25
0.84
['10 x 5 mm.', 'no response.', '14 mm.', '19 x 6 mm.', '19 mm.', '24 x 8 mm.', '11mm.', '8-9.2 mm.', '18.49-20 mm.', '32 mm.', '25x14 mm.', '13 mm.', '1cm.mm.', '9mm.', '9mm.', '6-7 mm.', '9 mm.', '12mm.', '20x8mm.', '17 mm.', '12 x 12 mm.', '24 x 20 mm.', '13 x 7 x 12 mm.', '10 x 5mm.', '3 cm.mm.'] ['10 x 5 mm.', 'no response.', '14 mm.', '19 x 6 mm.', '19 mm.', '24 x 8 mm.', '11mm.', '8-9.2 mm.', '18.49-20 mm.', '32 mm.', '25x14 mm.', '13 mm.', '1cm.', '9mm.', '20x14 mm.', '6-7 mm.', '9 mm.', '5mm.', '20x8mm.', '17 mm.', '12 x

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Examples: 25
Token F1: 0.8433
Exact Match: 0.8400
ROUGE-L: 0.8987
BERTScore F1: 0.9603
BERTScore R: 0.9565


In [9]:
AGE
0.9952830188679245
TIPO
0.9669811320754716
TECNICA
0.9858490566037735
FAMILY
0.9838709677419355
HISTORY
0.9139784946236559
SYMTOMATIC
0.9516129032258065
PROSTHESIS
0.989247311827957
BIRADS
0.9838709677419355
DENSITY_MAMMO
0.9623655913978495
DENSITY_ECHO
0.9354838709677419
LYMPH_SUSPICIOUS
0.9838709677419355
NODULES_ECHO_NUM
0.8225806451612904
NODULES_ECHO_SHAPE
0.926829268292683
NODULES_ECHO_MARGIN
0.8658536585365854
NODULES_ECHO_ECHOGENICITY
0.926829268292683
NODULES_ECHO_LOCATION
0.6585365853658537
NODULES_ECHO_SIZE
0.9146341463414634
NODULES_ECHO_KNOWN
0.7439024390243902
NODULES_ECHO_STABLE
0.7727272727272727

NameError: name 'AGE' is not defined

In [11]:

model_name="biogpt"+"1e57"+"no_copy"+"description"+"16batch"+"final_paper"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model_name}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
001-323-702-20221219-120748.390_age no. 
001-550-138-20230428-094656_age no. 
001-550-138-20231207-120430_age no. 
001-689-096-20230529-131355_age no. 
002-978-919-20230417-112452.015000_age no. 
008-536-472-20211122-103547_age no. 
010-422-670-20231023-154752_age 29. 
021-170-050-20240516-125540_age no. 
031-241-706-20231120-171306_age 48. 
032-339-068-20240311-165524_age 70. 
033-205-714-20240123-190445.901_age no. 
033-706-581-20230627-134123.180_age 18. 
036-688-025-20230313-135202_age no. 
037-052-626-20230529-120836.312_age no. 
039-107-431-20230717-112910_age no. 
039-717-776-20220627-123733.687_age no. 
040-885-712-20240226-170600_age 44. 
043-372-637-20220620-135102_age no. 
046-630-291-20230308-102622_age no. 
046-630-291-20230619-112356_age no. 
046-630-291-20230626-174935_age 38. 
049-148-620-20240419-163052.629_age no. 
052-661-084-20220314-132339_age no. 
056-956-396-20220509-111549.140_age no. 
058-197-862-20230511-152451_age no. 
060-049-218-20230717-130657.218_age 

In [5]:
model_name="biogpt"+"1e57"+"no_copy"+"description"+"16batch"+"final"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model_name}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
181-536-521-20221217-101331_age 46. no.
0.9952830188679245
TIPO
076-969-379-20240229-090526.971_tipo nodal staging ultrasound report. normal control or revision report.
087-932-399-20240520-152923_tipo biopsy report. nodal staging ultrasound report.
172-442-193-20230724-114130.406_tipo biopsy report. normal control or revision report.
278-883-630-20240227-153842.710_tipo biopsy report. nodal staging ultrasound report.
331-153-519-20230703-152749_tipo biopsy report. normal control or revision report.
654-622-062-20230529-154932_tipo biopsy report. nodal staging ultrasound report.
800-359-086-20230516-105151.599_tipo nodal staging ultrasound report. normal control or revision report.
0.9669811320754716
TECNICA
153-234-011-20230824-150145.697_tecnica mammography and ultrasound. only ultrasound study.
209-923-502-20230315-172153_tecnica only mammography study. mammography and ultrasound.
286-892-882-20240401-153506_tecnica only ultrasound study. mammography and ultrasound.
800-359-086-

In [5]:
model_name="biogpt"+"21e51e57"+"no_copy"+"description"+"16batch"+"final"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model_name}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo!="nodules_echo_size":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
181-536-521-20221217-101331_age 46. no.
0.9952830188679245
TIPO
076-969-379-20240229-090526.971_tipo nodal staging ultrasound report. normal control or revision report.
087-932-399-20240520-152923_tipo biopsy report. normal control or revision report.
172-442-193-20230724-114130.406_tipo biopsy report. normal control or revision report.
278-883-630-20240227-153842.710_tipo biopsy report. normal control or revision report.
286-892-882-20240401-153506_tipo biopsy report. normal control or revision report.
331-153-519-20230703-152749_tipo biopsy report. normal control or revision report.
654-622-062-20230529-154932_tipo biopsy report. nodal staging ultrasound report.
800-359-086-20230516-105151.599_tipo nodal staging ultrasound report. normal control or revision report.
0.9622641509433962
TECNICA
195-203-909-20230615-120646_tecnica only ultrasound study. mammography and ultrasound.
209-923-502-20230315-172153_tecnica only mammography study. mammography and ultrasound.
286-892-882-2024

In [5]:
model="biogpt"+"5e65e63"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo=="history" or tipo=="family" or tipo=="symtomatic" or tipo=="density_echo" or tipo=="nodules_echo_echogenicity" or tipo=="nodules_echo_margin" or tipo=="nodules_echo_shape"  or tipo=="nodules_echo_location":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
0.9952830188679245
TIPO
0.0
TECNICA
0.9764150943396226
FAMILY
0.9946236559139785
HISTORY
0.9032258064516129
SYMTOMATIC
0.946236559139785
PROSTHESIS
0.9946236559139785
BIRADS
0.9838709677419355
DENSITY_MAMMO
0.978494623655914
CALCIFICATIONS_BENIGN


FileNotFoundError: [Errno 2] No such file or directory: 'Generativos/results_dic_calcifications_benign/biogpt5e65e63.pkl'

In [10]:
model="biogpt1e55e63"
for tipo in questions:
    print(tipo.upper())
    with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
            truth=pickle.load(file)
    with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
            output=pickle.load(file)
    if tipo=="history" or tipo=="family" or tipo=="symtomatic" or tipo=="density_echo" or tipo=="nodules_echo_echogenicity" or tipo=="nodules_echo_margin" or tipo=="nodules_echo_shape"  or tipo=="nodules_echo_location":
        print(calculate_acc(output,truth,point=True))
    elif tipo=="nodules_echo_size":
        print(calculate_acc(output,truth,point="mm"))
    else:
        print(calculate_acc(output,truth))

AGE
0.9952830188679245
TIPO
0.9669811320754716
TECNICA
0.9858490566037735
FAMILY
0.9838709677419355
HISTORY
0.9139784946236559
SYMTOMATIC
0.9516129032258065
PROSTHESIS
0.989247311827957
BIRADS
0.9838709677419355
DENSITY_MAMMO
0.9623655913978495
DENSITY_ECHO
0.9354838709677419
LYMPH_SUSPICIOUS
0.9838709677419355
NODULES_ECHO_NUM
0.8225806451612904
NODULES_ECHO_SHAPE
0.926829268292683
NODULES_ECHO_MARGIN
0.8658536585365854
NODULES_ECHO_ECHOGENICITY
0.926829268292683
NODULES_ECHO_LOCATION
0.6585365853658537
NODULES_ECHO_SIZE
0.9146341463414634
NODULES_ECHO_KNOWN
0.7439024390243902
NODULES_ECHO_STABLE
0.7727272727272727


In [5]:
calculate_acc(biogpt_age,truth_age)

181-536-521-20221217-101331 46 no


0.9952830188679245

In [6]:
calculate_acc(biogpt_tipo,truth_tipo)

001-550-138-20231207-120430 no yes
286-892-882-20240401-153506 no yes
331-153-519-20230703-152749 no yes
534-723-539-20231013-165228.835 no yes


0.9811320754716981

In [7]:
calculate_acc(biogpt_tecnica,truth_tecnica)

049-148-620-20240419-163052.629 mammography and ultrasound only ultrasound study
153-234-011-20230824-150145.697 mammography and ultrasound only ultrasound study
154-571-251-20230928-113422 only ultrasound study mammography and ultrasound
181-536-521-20221217-101331 mammography and ultrasound only ultrasound study
195-203-909-20230615-120646 only ultrasound study mammography and ultrasound
209-923-502-20230315-172153 only mammography study mammography and ultrasound
227-053-436-20230714-113137 mammography and ultrasound only ultrasound study
286-892-882-20240401-153506 only ultrasound study only mammography study
800-359-086-20230522-154052 only mammography study mammography and ultrasound


0.9575471698113207